In [2]:
print("")
print("cgi-cov-gff_lk1.py")


cgi-cov-gff_lk1.py


In [3]:
#
import sys
sys.path.append("../dep")
from libs import *

In [4]:
a_contig = 'chr23'
a_window = 42

In [5]:
# # Collect args
# parser = argparse.ArgumentParser(description='sample')
# parser.add_argument('--contig', action="store", dest='contig', default=0)
# parser.add_argument('--window', action="store", dest='window', default=0)
# args = parser.parse_args()

In [6]:
#
dir1 = '../out1/gffdb_split/'
dir2 = '../out2/'+str(a_contig)+'/cgicov/'

In [7]:
#
print("")
print('w: '+str(a_window))


w: 42


In [10]:
print("annotation...")
# Import annotation
gdf = pd.read_csv(dir1+'gd-s_con-'+str(a_contig)+'_w'+str(a_window)+'.csv')
# Extract info out of attribute "att" column
nam = []
for s in range(len(gdf["att"])):
    ss = gdf["att"][s]
    if "Name" in ss:
        for i in ss.split(","):
            if "Name" in i:
                ii = i
                ii = ii.replace('"Name":', '')
                ii = ii.replace('["', '')
                ii = ii.replace('"]', '')
                nam.append(ii)
    else:
        nam.append("")
gdf['name'] = nam
# Subset
gdf = gdf[gdf['featuretype']=='gene'] # 
gdf = gdf.drop(columns='att')
gdf = gdf[gdf['name']!='']
gdf = gdf.reset_index(drop=True)
gdf

annotation...


,chrom,id,featuretype,strand,start,end,name
0,chr23,gene-LIN52,gene,-,42021782,42093362,LIN52
1,chr23,gene-ALDH6A1,gene,+,42111153,42155977,ALDH6A1
2,chr23,gene-BBOF1,gene,-,42156230,42181421,BBOF1
3,chr23,gene-LOC119944548,gene,+,42160667,42170796,LOC119944548
4,chr23,gene-LOC119944588,gene,+,42190809,42200168,LOC119944588
5,chr23,gene-LOC119944754,gene,-,42221673,42267487,LOC119944754
6,chr23,gene-LOC119944753,gene,+,42282492,42324713,LOC119944753
7,chr23,gene-COQ6,gene,-,42326814,42348802,COQ6
8,chr23,gene-LOC119944490,gene,+,42350589,42356725,LOC119944490
9,chr23,gene-FAM161B,gene,+,42356919,42370263,FAM161B


In [11]:
print("cgicov...")
# import cgi cov
cdf = pd.read_csv(dir2+'cgi-cov_con-'+str(a_contig)+'_w'+str(a_window)+'.csv')
ww = []
for w in range(len(cdf['rid'])):
    ww.append(w)
cdf['w'] = ww

cgicov...


In [12]:
cdf

,rid,cg_mean,cg_max,cgi_num,cgi,xst,xen,cg_pct,tv_n,tp_m,tp_s,tm_t,tu_t,w
0,chr23,0.828397,15.2,211,c,42021000,42021999,3.4,116,83.944275,12.569616,2026,380,0
1,chr23,0.828397,15.2,211,c,42022000,42022999,5.4,112,85.260599,12.934149,2306,376,1
2,chr23,0.828397,15.2,211,c,42023000,42023999,8.2,167,90.262563,8.389347,3346,365,2
3,chr23,0.828397,15.2,211,c,42039000,42039999,4.0,109,82.582588,19.821220,2007,438,3
4,chr23,0.828397,15.2,211,c,42040000,42040999,3.8,115,78.600680,19.790987,1670,423,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,chr23,0.828397,15.2,211,c,42927000,42927999,4.6,179,90.307936,11.678325,2702,297,72
73,chr23,0.828397,15.2,211,c,42928000,42928999,8.1,174,91.463120,16.518945,1300,108,73
74,chr23,0.828397,15.2,211,c,42929000,42929999,7.5,186,88.849685,17.761195,2338,280,74
75,chr23,0.828397,15.2,211,c,42950000,42950999,4.7,140,88.465689,16.573062,1019,128,75


In [13]:
print("find genes...")
# Find genes inside each window
mm = 5000
pp = 5000
tot_insi = []
sta_insi = []
end_insi = []
prox_m = []
prox_p = []
span_gen = []
for w in cdf['w']:
    for g in range(len(gdf['name'])):
        if cdf['xst'].iloc[w] <= gdf['start'].iloc[g] <= cdf['xen'].iloc[w]:
            if cdf['xst'].iloc[w] <= gdf['end'].iloc[g] <= cdf['xen'].iloc[w]:
                tot_insi.append([w, gdf['name'].iloc[g], gdf['strand'].iloc[g],
                                 gdf['start'].iloc[g], gdf['end'].iloc[g]])
            else:
                sta_insi.append([w, gdf['name'].iloc[g], gdf['strand'].iloc[g],
                                 gdf['start'].iloc[g], gdf['end'].iloc[g]])
        elif cdf['xst'].iloc[w] <= gdf['end'].iloc[g] <= cdf['xen'].iloc[w]:
            end_insi.append([w, gdf['name'].iloc[g], gdf['strand'].iloc[g],
                             gdf['start'].iloc[g], gdf['end'].iloc[g]])
        if cdf['xst'].iloc[w] <= gdf['start'].iloc[g] - mm <= cdf['xen'].iloc[w]:
            prox_m.append([w, gdf['name'].iloc[g], gdf['strand'].iloc[g],
                           gdf['start'].iloc[g], gdf['end'].iloc[g]])
        elif cdf['xst'].iloc[w] <= gdf['end'].iloc[g] + pp <= cdf['xen'].iloc[w]:
            prox_p.append([w, gdf['name'].iloc[g], gdf['strand'].iloc[g],
                           gdf['start'].iloc[g], gdf['end'].iloc[g]])
        if gdf['start'].iloc[g] < cdf['xst'].iloc[w]:
            if gdf['end'].iloc[g] > cdf['xen'].iloc[w]:
                span_gen.append([w, gdf['name'].iloc[g], gdf['strand'].iloc[g],
                                 gdf['start'].iloc[g], gdf['end'].iloc[g]])

find genes...


In [14]:
# Add empty row to ensure use
#
tot_insi.append(['','','','',''])
df_tot = pd.DataFrame(tot_insi)
df_tot.columns = ['w', 'name', 'strand', 'start', 'end']
#
sta_insi.append(['','','','',''])
df_sta = pd.DataFrame(sta_insi)
df_sta.columns = ['w', 'name', 'strand', 'start', 'end']
#
end_insi.append(['','','','',''])
df_end = pd.DataFrame(end_insi)
df_end.columns = ['w', 'name', 'strand', 'start', 'end']
#
prox_m.append(['','','','',''])
df_xm = pd.DataFrame(prox_m)
df_xm.columns = ['w', 'name', 'strand', 'start', 'end']
#
prox_p.append(['','','','',''])
df_xp = pd.DataFrame(prox_p)
df_xp.columns = ['w', 'name', 'strand', 'start', 'end']
#
span_gen.append(['','','','',''])
df_sp = pd.DataFrame(span_gen)
df_sp.columns = ['w', 'name', 'strand', 'start', 'end']

In [15]:
print("get genes by class...")
# Get genes from each class for each window
g_w = []
g_nam = []
g_str = []
g_sta = []
g_end = []
g_des = []
for w in cdf['w']:
    if len(df_tot[df_tot['w']==w]['w']) != 0:
        g_w.append(df_tot[df_tot['w']==w]['w'].iloc[0])
        g_nam.append(df_tot[df_tot['w']==w]['name'].iloc[0])
        g_str.append(df_tot[df_tot['w']==w]['strand'].iloc[0])
        g_sta.append(df_tot[df_tot['w']==w]['start'].iloc[0])
        g_end.append(df_tot[df_tot['w']==w]['end'].iloc[0])
        g_des.append("tot_insi")
    if len(df_sta[df_sta['w']==w]['w']) != 0:
        g_w.append(df_sta[df_sta['w']==w]['w'].iloc[0])
        g_nam.append(df_sta[df_sta['w']==w]['name'].iloc[0])
        g_str.append(df_sta[df_sta['w']==w]['strand'].iloc[0])
        g_sta.append(df_sta[df_sta['w']==w]['start'].iloc[0])
        g_end.append(df_sta[df_sta['w']==w]['end'].iloc[0])
        g_des.append("sta_insi")
    if len(df_end[df_end['w']==w]['w']) != 0:
        g_w.append(df_end[df_end['w']==w]['w'].iloc[0])
        g_nam.append(df_end[df_end['w']==w]['name'].iloc[0])
        g_str.append(df_end[df_end['w']==w]['strand'].iloc[0])
        g_sta.append(df_end[df_end['w']==w]['start'].iloc[0])
        g_end.append(df_end[df_end['w']==w]['end'].iloc[0])
        g_des.append("end_insi")
    if len(df_xm[df_xm['w']==w]['w']) != 0:
        g_w.append(df_xm[df_xm['w']==w]['w'].iloc[0])
        g_nam.append(df_xm[df_xm['w']==w]['name'].iloc[0])
        g_str.append(df_xm[df_xm['w']==w]['strand'].iloc[0])
        g_sta.append(df_xm[df_xm['w']==w]['start'].iloc[0])
        g_end.append(df_xm[df_xm['w']==w]['end'].iloc[0])
        g_des.append("prox_m")
    if len(df_xp[df_xp['w']==w]['w']) != 0:
        g_w.append(df_xp[df_xp['w']==w]['w'].iloc[0])
        g_nam.append(df_xp[df_xp['w']==w]['name'].iloc[0])
        g_str.append(df_xp[df_xp['w']==w]['strand'].iloc[0])
        g_sta.append(df_xp[df_xp['w']==w]['start'].iloc[0])
        g_end.append(df_xp[df_xp['w']==w]['end'].iloc[0])
        g_des.append("prox_p")
    if len(df_sp[df_sp['w']==w]['w']) != 0:
        g_w.append(df_sp[df_sp['w']==w]['w'].iloc[0])
        g_nam.append(df_sp[df_sp['w']==w]['name'].iloc[0])
        g_str.append(df_sp[df_sp['w']==w]['strand'].iloc[0])
        g_sta.append(df_sp[df_sp['w']==w]['start'].iloc[0])
        g_end.append(df_sp[df_sp['w']==w]['end'].iloc[0])
        g_des.append("span_gen")
ann = pd.DataFrame([g_w, g_nam, g_str, g_sta, g_end, g_des]).T
ann.columns = ['g_w', 'g_nam', 'g_str', 'g_sta', 'g_end', 'g_des']

get genes by class...


In [16]:
ann

,g_w,g_nam,g_str,g_sta,g_end,g_des
0,0,LIN52,-,42021782,42093362,sta_insi
1,1,LIN52,-,42021782,42093362,span_gen
2,2,LIN52,-,42021782,42093362,span_gen
3,3,LIN52,-,42021782,42093362,span_gen
4,4,LIN52,-,42021782,42093362,span_gen
...,...,...,...,...,...,...
62,72,PAPLN,-,42932342,42990029,prox_m
63,72,NUMB,+,42819042,42928802,span_gen
64,73,NUMB,+,42819042,42928802,end_insi
65,75,PAPLN,-,42932342,42990029,span_gen


In [17]:
print("combine")
# Combine info for each window
c_w = []
c_len = []
c_nam = []
c_str = []
c_sta = []
c_end = []
c_des = []
for i in ann['g_w'].unique():
    c_w.append(i)
    c_len.append(len(ann[ann['g_w']==i]['g_nam']))
    c_nam.append(list(ann[ann['g_w']==i]['g_nam']))
    c_str.append(list(ann[ann['g_w']==i]['g_str']))
    c_sta.append(list(ann[ann['g_w']==i]['g_sta']))
    c_end.append(list(ann[ann['g_w']==i]['g_end']))
    c_des.append(list(ann[ann['g_w']==i]['g_des']))
cann = pd.DataFrame([c_w, c_len, c_nam, c_str, c_sta, c_end, c_des]).T
cann.columns = ['c_w', 'c_len', 'c_nam', 'c_str', 'c_sta', 'c_end', 'c_des']

combine


In [18]:
# Fill in missing values
caw = list(cann['c_w'])
cdw = list(cdf['w'])
m_w = list(set(cdw) - set(caw))
m_len = [''] * len(m_w)
m_nam = [''] * len(m_w)
m_str = [''] * len(m_w)
m_sta = [''] * len(m_w)
m_end = [''] * len(m_w)
m_des = [''] * len(m_w)
mdf = pd.DataFrame([m_w, m_len, m_nam, m_str, m_sta, m_end, m_des]).T
mdf.columns = ['c_w', 'c_len', 'c_nam', 'c_str', 'c_sta', 'c_end', 'c_des']
gen = pd.concat([mdf, cann])
gen = gen.sort_values(by='c_w')
gen.rename(columns={'c_w':'w'}, inplace=True)

In [19]:
print("merge...")
# Merge with original dataset
cdf.head()
mer = cdf.merge(gen, how='inner', on='w')
#mer.head()
print("save data")
mer.to_csv('../out2/'+str(a_contig)+'/anno/cvg_con-'+str(a_contig)+'_w'+str(a_window)+'.csv', index=False)

merge...
save data


In [20]:
mer

,rid,cg_mean,cg_max,cgi_num,cgi,xst,xen,cg_pct,tv_n,tp_m,tp_s,tm_t,tu_t,w,c_len,c_nam,c_str,c_sta,c_end,c_des
0,chr23,0.828397,15.2,211,c,42021000,42021999,3.4,116,83.944275,12.569616,2026,380,0,1,[LIN52],[-],[42021782],[42093362],[sta_insi]
1,chr23,0.828397,15.2,211,c,42022000,42022999,5.4,112,85.260599,12.934149,2306,376,1,1,[LIN52],[-],[42021782],[42093362],[span_gen]
2,chr23,0.828397,15.2,211,c,42023000,42023999,8.2,167,90.262563,8.389347,3346,365,2,1,[LIN52],[-],[42021782],[42093362],[span_gen]
3,chr23,0.828397,15.2,211,c,42039000,42039999,4.0,109,82.582588,19.821220,2007,438,3,1,[LIN52],[-],[42021782],[42093362],[span_gen]
4,chr23,0.828397,15.2,211,c,42040000,42040999,3.8,115,78.600680,19.790987,1670,423,4,1,[LIN52],[-],[42021782],[42093362],[span_gen]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,chr23,0.828397,15.2,211,c,42927000,42927999,4.6,179,90.307936,11.678325,2702,297,72,2,"[PAPLN, NUMB]","[-, +]","[42932342, 42819042]","[42990029, 42928802]","[prox_m, span_gen]"
73,chr23,0.828397,15.2,211,c,42928000,42928999,8.1,174,91.463120,16.518945,1300,108,73,1,[NUMB],[+],[42819042],[42928802],[end_insi]
74,chr23,0.828397,15.2,211,c,42929000,42929999,7.5,186,88.849685,17.761195,2338,280,74,,,,,,
75,chr23,0.828397,15.2,211,c,42950000,42950999,4.7,140,88.465689,16.573062,1019,128,75,1,[PAPLN],[-],[42932342],[42990029],[span_gen]
